In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/ehsan/PycharmProjects/TextSummarizer/Text-Summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    root_dir : Path
    data_path : Path
    model_path : Path
    tokenizer_path : Path
    metric_filename : Path
    


In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_dirs

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_dirs([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> EvaluationConfig:
        config  = self.config.model_evaluation
        create_dirs([config.root_dir])

        evaluation_config = EvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_filename = config.metric_filename
        )

        return evaluation_config

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

In [8]:
class ModelEvaluation:
    def __init__(self, config: EvaluationConfig) -> None:
        self.config = config

    
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Yield successive n-sized chunks from l."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i:i + batch_size]

    def calculate_metrics_on_test_ds(self, dataset, metric, model, tokenizer,
                                     batch_size=16, device='cuda' if torch.cuda.is_available() else 'cpu',
                                     column_text='article',
                                     column_summary='summary'):
        article_batch = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batch = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batch, target_batch), total=len(article_batch)):
            
            inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                               paddings="max_length", return_tensors="pt")
            
            summaries = model.generate(inputs_ids=inputs['inputs_ids'].to(device),
                                       attention_mask=inputs['attention_mask'].to(device),
                                       length_penalty=0.8, num_beams=8, max_length=128)
            

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                                  clean_up_tokenization_spaces=True)
                                 for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries] 

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

            return metric.compute()
        
        def evaluate(self):
            device = "cuda" if torch.cuda.is_available() else "cpu"
            tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
            model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

            dataset_samsum_pt = load_from_disk(self.config.data_path)

            rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

            rouge_metric = load_metric('rouge')

            score = self.calculate_metric_on_test_ds(dataset_samsum_pt['test'][:10], rouge_metric,
                                                     model, 2, tokenizer, device)
            
            rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

            df = pd.DataFrame(rouge_dict, index=['pegasus'] )
            df.to_csv(self.config.metric_filename, index=False)

In [ ]:
try: 
    config = ConfigurationManager()
    evaluation_config = config.get_model_evaluation_config()
    evaluation = ModelEvaluation(config=evaluation_config)
    evaluation.evaluate()

except Exception as e:
     raise e
